# Load Data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium
from folium import plugins
from folium.plugins import HeatMap

plt.rcParams["font.family"] = "Garamond"
plt.rcParams['axes.facecolor'] = "#E8ECEB"
plt.rcParams['axes.facecolor'] = "#E8ECEB"

cols_of_interest = ['Opened', 'Closed', 'Updated', 'Status', 'Status Notes', 'Responsible Agency', 'Category', 'Request Type', 'Request Details', 'Address', 'Supervisor District', 'Neighborhood', 'Latitude', 'Longitude', 'Source', 'Media URL']

data = pd.read_csv('/Users/Gustav/Downloads/311_Cases0105.csv', usecols=cols_of_interest)

/var/folders/1j/15c8wxfx0rz57_sf9j19y3fh0000gp/T/ipykernel_34169/3145598073.py:15: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/Gustav/Downloads/311_Cases0105.csv', usecols=cols_of_interest)


In [3]:
# convert to datetime
data['Opened'] = pd.to_datetime(data['Opened'])
# create a column called yearmonth that contain the year and month of the opened date
data['yearmonth'] = data['Opened'].apply(lambda x: str(x.year) + '-' + str(x.month) + '-01')

# take dataframe and get a count of every Request Details value summed by month and year
df = data[(data['Category']=="Street and Sidewalk Cleaning")]
# remove rows where latitude or longitude is null or zero
df = df[(df['Latitude']!=0) | (df['Longitude']!=0)]

### Fix Request details types that have different names but are the same
## Garbage and Debris
df['Request Details'] = df['Request Details'].replace(['Other Loose Garbage', 'Other Loose Garbage / Debris', 'Auto_accident_debris',
                                                       'Garbage and Debris - N/A','Garbage and Debris - {{nature_of_request.label}}',
                                                        'Garbage and Debris - {{request_type.label}}'],
                                                        "Loose Garbage and Debris")
## Overflowing city garbage can
df['Request Details'] = df['Request Details'].replace(['City_garbage_can_overflowing', 'Overflowing city receptacle or dumpster',
                                                        'Overflowing_City_Receptacle_or_Dumpster'], "Overflowing City Garbage Can")
## Illegal dumping
df['Request Details'] = df['Request Details'].replace(['Illegal Dumping', 'Illegal dumping', 'Illegal_Dumping'], "Illegal Dumping")
## Furniture, mattresses and refrigerators
df['Request Details'] = df['Request Details'].replace(['Furniture', 'Mattress', 'Refrigerator'], "Furniture, Mattresses and Refrigerators")
## Sidewalk or street cleaning
df['Request Details'] = df['Request Details'].replace(['Sidewalk_Cleaning', 'Sidewalk cleaning', 'Street_Cleaning', 'Street cleaning'], "Sidewalk or Street Cleaning")
## Missed Route Mechanical Sweeping
df['Request Details'] = df['Request Details'].replace(['Missed_Route_Mechanical_Sweeping', 'Missed route - mechanical sweeping'], "Missed Route Mechanical Sweeping")


### Rename some values to be more pretty
## Transit_shelter_platform
df['Request Details'] = df['Request Details'].replace(['Transit_shelter_platform'], "Transit Shelter Platform")
## Christmas tree
df['Request Details'] = df['Request Details'].replace(['Christmas_tree'], "Christmas Tree")
## Event_parade_mess
df['Request Details'] = df['Request Details'].replace(['Event_parade_mess'], "Event Parade Mess")
## Missed_TrashRecycleCompost_Collection
df['Request Details'] = df['Request Details'].replace(['Missed_TrashRecycleCompost_Collection'], "Missed Trash Recycle Compost Collection")
## Medical_waste
df['Request Details'] = df['Request Details'].replace(['Medical_waste'], "Medical Waste")
## GarbageTransit
df['Request Details'] = df['Request Details'].replace(['GarbageTransit'], "Garbage Transit")
## OtherTransit
df['Request Details'] = df['Request Details'].replace(['OtherTransit'], "Other Transit")
## PostAbatementInspection
df['Request Details'] = df['Request Details'].replace(['PostAbatementInspection'], "Post Abatement Inspection")


# print the unique values of Request Details and their counts
df['Request Details'].value_counts()

Loose Garbage and Debris                   810935
Furniture, Mattresses and Refrigerators    540974
Boxed or Bagged Items                      287794
Human or Animal Waste                      257784
Overflowing City Garbage Can               142716
Electronics                                 60910
Glass                                       52834
Needles                                     51013
Liquids - Oil/Paint/Other                   27098
Transit Shelter Platform                    15106
Christmas Tree                              10101
Missed Route Mechanical Sweeping             6277
Paper Spill                                  5260
Tires                                        4800
Hazardous Waste Containers                   4442
Illegal Dumping                              1638
Event Parade Mess                             951
Car Batteries                                 390
Missed Trash Recycle Compost Collection       389
Medical Waste                                 301


In [4]:
df['Year'] = df['Opened'].dt.year

df.head()

,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Supervisor District,Neighborhood,Latitude,Longitude,Source,Media URL,yearmonth,Year
54,2008-11-15 08:07:43,11/19/2008 02:06:59 PM,11/19/2008 02:06:59 PM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Loose Garbage and Debris,Intersection of CAMBRIDGE ST and GLADSTONE DR,11.0,Mission Terrace,37.731464,-122.420189,Phone,NaN,2008-11-01,2008
59,2008-11-14 14:44:35,11/15/2008 06:07:03 AM,11/15/2008 06:07:03 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Loose Garbage and Debris,Intersection of 17TH AVE and CABRILLO ST,1.0,Inner Richmond,37.774830,-122.475754,Phone,NaN,2008-11-01,2008
65,2008-11-10 08:10:22,11/12/2008 03:06:59 PM,11/12/2008 03:06:59 PM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Loose Garbage and Debris,"28 13TH ST, SAN FRANCISCO, CA, 94103",6.0,South of Market,37.769745,-122.413742,Phone,NaN,2008-11-01,2008
71,2008-11-05 12:42:54,11/06/2008 06:06:57 AM,11/06/2008 06:06:57 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Loose Garbage and Debris,Intersection of 18TH ST and CASTRO ST,8.0,Castro,37.760887,-122.434998,Phone,NaN,2008-11-01,2008
77,2008-11-03 12:19:28,11/06/2008 06:07:10 AM,11/06/2008 06:07:10 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,"Furniture, Mattresses and Refrigerators","5401 MISSION ST, SAN FRANCISCO, CA, 94112",11.0,Crocker Amazon,37.713654,-122.443405,Phone,NaN,2008-11-01,2008


In [12]:
import pickle

with open('data.pkl', 'wb') as f:
    pickle.dump(df, f)

# Loading Shortcut

In [20]:
!pwd

/Users/Gustav/Desktop/DesktopV00183/DTU/2nd/SocialDA&Viz.nosync/SocialDataFinalProject/coding


In [32]:
import plotly.graph_objs as go

yearly_counts = df.groupby('Year').size()
yearly_counts = yearly_counts.drop(2023)

hover_text = [f"<b>{val/1000:.1f}k</b> <extra></extra>" if val >= 1000 else str(val) for val in yearly_counts.values]

fig = go.Figure(go.Bar(
    x=yearly_counts.index,
    y=yearly_counts.values,
    hovertemplate=hover_text,
    marker_color='#487C4F',
    marker_line_color='black',
    marker_line_width=1,
    hoverlabel=dict(bgcolor="gold"),
))

# Set x-axis ticks to show all years
fig.update_layout(xaxis=dict(dtick=1))

fig.update_layout(height = 550,
                    width = 950,
                    autosize=True,
                    paper_bgcolor= "#E8ECEB",
                    margin={"r":10,"t":0,"l":10,"b":0},
                    font_family = "Garamond",
                    font_size = 18,
                    xaxis_title="Year",
                    yaxis_title="Number of Requests",
                    )
fig.update_layout(
    plot_bgcolor='#E8ECEB'
)

fig.show()
fig.write_html('/Users/Gustav/Desktop/DesktopV00183/DTU/2nd/SocialDA&Viz.nosync/SocialDataFinalProject/assets/images/BarChart.html')
